In [1]:
import os
import pandas as pd
import glob
import json
import datetime as dt
import xml.etree.ElementTree as ET
import logging
import time

In [2]:
PARQUET_DIRECTORY = './'
PARQUET_FILE_THRESHOLD = 10
POLLING_INTERVAL_SECONDS = 60 

In [10]:
def parse_parquet_directory(dir_path):
    logging.info(f'Starting to parse Parquet files in directory {dir_path}.')
    try:
        # Find all parquet files in the directory
        parquet_files = glob.glob(f'{dir_path}/*.parquet')
        logging.info('Found %s parquet files.', len(parquet_files))

        all_data = []
        for file_path in parquet_files:
            logging.info('Parsing file: %s', file_path)
            df = pd.read_parquet(file_path)

            # Iterate over the 'value' column and parse the XML content
            for xml_content in df['value']:
                xml_content = xml_content.replace('\\"', '"').strip('"')
                parsed_data = parse_xml(xml_content)
                all_data.extend(parsed_data)

        # Convert the list of dictionaries to a dataframe
        all_data_df = pd.DataFrame(all_data)
        logging.info(f'Finished parsing Parquet files in directory {dir_path}.')
        return all_data_df
    except Exception as e:
        logging.exception(f"Error occurred while parsing parquet directory: {e}")


def parse_xml(xml_content):
    logging.info('Starting to parse XML content.')
    try:
        root = ET.fromstring(xml_content)

        results = []
        for ur in root.findall('{http://www.thalesgroup.com/rtti/PushPort/v16}uR'):
            for ts in ur.findall('{http://www.thalesgroup.com/rtti/PushPort/v16}TS'):
                result = {
                    'ts' : root.attrib.get('ts'),
                    'rid': ts.attrib.get('rid'),
                    'uid': ts.attrib.get('uid'),
                    'ssd': ts.attrib.get('ssd'),
                    'updateOrigin': ur.attrib.get('updateOrigin'),
                }
                locations = []
                for location in ts.findall('{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}Location'):
                    location_data = {
                        'tpl': location.attrib.get('tpl'),
                        'wtp': location.attrib.get('wtp'),
                        'wta': location.attrib.get('wta'),
                        'wtd': location.attrib.get('wtd'),
                        'pta': location.attrib.get('pta'),
                        'ptd': location.attrib.get('ptd'),
                    }
                    # note: might want to add 'pass' here and around 112 below. it might occasionally appear
                    # along with/instead of arr/dep.
                    for tag in ['pass', 'arr', 'dep', 'plat', 'length']:
                        tag_element = location.find("{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}" + tag)
                        print(tag_element.text)
                        if tag_element is not None:
                            location_data[tag] = tag_element.text or tag_element.attrib
                    locations.append(location_data)
                result['locations'] = locations
                results.append(result)
        logging.info('Finished parsing XML content.')
        return results
    except Exception as e:
        logging.exception(f"Error occurred while parsing XML data: {e}")


def transform(df):
    logging.info('Starting to transform data.')
    try:
        # explode the list column into separate rows
        exploded_df = df.explode('locations')

        # create separate columns from dictionary keys
        final_df = exploded_df['locations'].apply(pd.Series)

        # merge with the original df
        final_df = pd.concat([exploded_df.drop('locations', axis=1), final_df], axis=1)

        # reset the index of final_df
        final_df = final_df.reset_index()

        # Convert time columns to datetime.time
        time_cols = ['wtp', 'wta', 'wtd']
        for col in time_cols:
            final_df[col] = final_df[col].apply(add_seconds)
            final_df[col] = pd.to_datetime(final_df['ssd'] + ' ' + final_df[col], format='%Y-%m-%d %H:%M:%S')

        final_df = final_df.loc[final_df['arr'].notnull() & final_df['dep'].notnull()]

        final_df['aat'] = final_df['arr'].apply(lambda d: d.get('et'))
        final_df['adt'] = final_df['dep'].apply(lambda d: d.get('et')) 

        final_df['a_delay'] = final_df['arr'].apply(lambda d: d.get('delayed'))
        final_df['d_delay'] = final_df['dep'].apply(lambda d: d.get('delayed'))

        # Convert 'delayed' to boolean
        final_df['a_delay'] = final_df['a_delay'].map({'true': True, None: False})
        final_df['d_delay'] = final_df['d_delay'].map({'true': True, None: False})

        time_cols = ['pta', 'ptd', 'aat', 'adt']
        for col in time_cols:
            final_df[col] = pd.to_datetime(final_df['ssd'] + ' ' + final_df[col], format='%Y-%m-%d %H:%M')
        
        logging.info('Finished transforming data.')
        # Now, you can drop the original 'index' column if you want
        return final_df.drop(columns=['index', 'arr', 'dep'])\
                       .rename(columns={
                                'rid': 'route_id',
                                'uid': 'unique_id',
                                'ssd': 'service_start_date',
                                'updateOrigin': 'update_origin',
                                'tpl': 'train_platform',
                                'wtp': 'working_time_pass',
                                'wta': 'working_time_arrival',
                                'wtd': 'working_time_departure',
                                'pta': 'planned_time_arrival',
                                'ptd': 'planned_time_departure',
                                'aat': 'actual_arrival_time',
                                'adt': 'actual_departure_time',
                                'plat': 'platform',
                                'length': 'train_length',
                                'et': 'estimated_time',
                                'src': 'source',
                                'at': 'actual_time',
                                'atClass': 'actual_time_class',
                                'a_delay': 'is_delayed_arrival',
                                'd_delay': 'is_delayed_departure',
                                'srcInst': 'source_instance',
                                'etmin': 'estimated_time_minutes'
                            })
    except Exception as e:
        logging.exception(f"Error occurred while transforming data: {e}")

In [11]:
df = parse_parquet_directory(PARQUET_DIRECTORY)

In [12]:
df

,rid,uid,ssd,updateOrigin,locations
0,202403287665473,L65473,2024-03-28,TD,"[{'tpl': 'REDBDGE', 'wtp': '17:15', 'wta': Non..."
1,202403287604061,L04061,2024-03-28,CIS,"[{'tpl': 'CNNBELL', 'wtp': None, 'wta': '17:33..."
2,202403287619099,L19099,2024-03-28,TD,"[{'tpl': 'STFD', 'wtp': None, 'wta': '17:27:30..."
3,202403287619099,L19099,2024-03-28,CIS,"[{'tpl': 'MRYLAND', 'wtp': '17:29:30', 'wta': ..."
4,202403286792956,C92956,2024-03-28,TD,"[{'tpl': 'HILLSID', 'wtp': None, 'wta': '17:30..."
5,202403287616064,L16064,2024-03-28,Trust,"[{'tpl': 'WLWYNN', 'wtp': None, 'wta': '17:30'..."
6,202403288703082,W03082,2024-03-28,CIS,"[{'tpl': 'THANETP', 'wtp': '17:27', 'wta': Non..."
7,202403288755124,W55124,2024-03-28,CIS,"[{'tpl': 'LARBERT', 'wtp': '17:33', 'wta': Non..."
8,202403287180165,G80165,2024-03-28,CIS,"[{'tpl': 'PADTLL', 'wtp': None, 'wta': None, '..."
9,202403287179899,G79899,2024-03-28,CIS,"[{'tpl': 'TWYFORD', 'wtp': None, 'wta': '21:05..."


In [29]:
new_df = pd.read_parquet('part-00000-758e553c-4215-4322-a054-9199fd01beed-c000.snappy.parquet')

In [30]:
all_data = []
for xml_content in new_df['value']:
    xml_content = xml_content.replace('\\"', '"').strip('"')
                # parsed_data = parse_xml(xml_content)

In [31]:
xml_content

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><Pport xmlns="http://www.thalesgroup.com/rtti/PushPort/v16" xmlns:ns2="http://www.thalesgroup.com/rtti/PushPort/Schedules/v3" xmlns:ns3="http://www.thalesgroup.com/rtti/PushPort/Schedules/v2" xmlns:ns4="http://www.thalesgroup.com/rtti/PushPort/Formations/v2" xmlns:ns5="http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3" xmlns:ns6="http://www.thalesgroup.com/rtti/PushPort/Formations/v1" xmlns:ns7="http://www.thalesgroup.com/rtti/PushPort/StationMessages/v1" xmlns:ns8="http://www.thalesgroup.com/rtti/PushPort/TrainAlerts/v1" xmlns:ns9="http://www.thalesgroup.com/rtti/PushPort/TrainOrder/v1" xmlns:ns10="http://www.thalesgroup.com/rtti/PushPort/TDData/v1" xmlns:ns11="http://www.thalesgroup.com/rtti/PushPort/Alarms/v1" xmlns:ns12="http://thalesgroup.com/RTTI/PushPortStatus/root_1" ts="2024-03-29T16:49:00.7564087Z" version="16.0"><uR updateOrigin="TD"><TS rid="202403298155120" uid="Q55120" ssd="2024-03-29"><ns5:Location tpl="BARNES" 

In [37]:
root = ET.fromstring(xml_content)
# results = []
# root.find('{http://www.thalesgroup.com/rtti/PushPort/v16}Pport')

In [38]:
print(root)

<Element '{http://www.thalesgroup.com/rtti/PushPort/v16}Pport' at 0x7f2350093560>


In [39]:
print(root.text)

None


In [47]:
ts = root.attrib.get('ts')

In [49]:
pd.to_datetime(ts)

Timestamp('2024-03-28 17:33:12.919408700+0000', tz='UTC')

In [46]:
results = []
for ur in root.findall('{http://www.thalesgroup.com/rtti/PushPort/v16}uR'):
    for ts in ur.findall('{http://www.thalesgroup.com/rtti/PushPort/v16}TS'):
        result = {
            'ts' : root.attrib.get('ts'),
            'rid': ts.attrib.get('rid'),
            'uid': ts.attrib.get('uid'),
            'ssd': ts.attrib.get('ssd'),
            'updateOrigin': ur.attrib.get('updateOrigin'),
        }
        locations = []
        for location in ts.findall('{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}Location'):
            location_data = {
                'tpl': location.attrib.get('tpl'),
                'wtp': location.attrib.get('wtp'),
                'wta': location.attrib.get('wta'),
                'wtd': location.attrib.get('wtd'),
                'pta': location.attrib.get('pta'),
                'ptd': location.attrib.get('ptd'),
            }
            # note: might want to add 'pass' here and around 112 below. it might occasionally appear
            # along with/instead of arr/dep.
            for tag in ['pass', 'arr', 'dep', 'plat', 'length']:
                tag_element = location.find("{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}" + tag)
                if tag_element is not None:
                    # print(tag_element.text)
                    # print(tag_element.attrib)
                    location_data[tag] = tag_element.text or tag_element.attrib

None
{'et': '17:35', 'wet': '17:34', 'src': 'Darwin'}
None
{'et': '17:35', 'src': 'Darwin'}
2
{'platsrc': 'A', 'conf': 'true'}


In [42]:
tag_element

In [43]:
test_elem = location.find("{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}" + "arr")
test_elem

<Element '{http://www.thalesgroup.com/rtti/PushPort/Forecasts/v3}arr' at 0x7f23500937e0>

In [45]:
test_elem.attrib

{'et': '17:35', 'wet': '17:34', 'src': 'Darwin'}